# MATH 189 Homework 3

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

# Import Data
#bottom,surface
water = pd.DataFrame([[0.43,0.415],
[0.266,0.238],
[0.567,0.39],
[0.531,0.41],
[0.707,0.605],
[0.716,0.609],
[0.651,0.632],
[0.589,0.523],
[0.469,0.411],
[0.723,0.612]], columns=['bottom', 'surface'])
#water = pd.read_csv('water.txt', encoding="UCS-2LE")
pottery = pd.read_csv('RBPottery.csv')

## PROBLEM 1

1 . Suppose we consider the zinc concentration in bottom water
and in surface water as two samples. Denote $\mu_1$ and $\mu_2$ the
underlying population means of the two samples. Test the null
and alternative hypotheses: 
$$H_{0}: \mu_1 = \mu_2\\
H_{1}: \mu_1 \neq \mu_2$$


In [10]:
display(water)
water.mean()

,bottom,surface
0,0.430,0.415
1,0.266,0.238
2,0.567,0.390
3,0.531,0.410
4,0.707,0.605
5,0.716,0.609
6,0.651,0.632
7,0.589,0.523
8,0.469,0.411
9,0.723,0.612


bottom     0.5649
surface    0.4845
dtype: float64

In [8]:
d = water.bottom - water.surface
t, p = stats.ttest_1samp(d, 0)
alpha = 0.05
if p < alpha:
    print('We reject the null hypothesis, the sample means are not equal.')
else:
    print('There is not enough evidence to reject the null hypothesis.')
{'T Statistic': t, 'p value': p}

We reject the null hypothesis, the sample means are not equal.


{'T Statistic': 4.8638127451351831, 'p value': 0.00089111545782254793}

2 . The two-sample test depends on the following four
assumptions.

1. The data from group $i$ has a common mean $\mu_i$
2. Homoskedasticity: The data from all groups have common
variance $\sigma^2$
3. Independence: The observations are independently
sampled. 
4. Normality: The data are normally distributed.

Pick one assumption from the above four and discuss what will
happen if this assumption is violated in practice. 

## PROBLEM 2

1 . Decompose the problem into multiple hypotheses testing: for
$k=1,...,9$, consider the null $H_0^{(k)}: \mu_1=\mu_2=\mu_3=\mu_4=\mu_5$ .
Calculate $\mathcal{F}$-statistics $\mathcal{F}^{(k)}(k=1,...,9)$ for these nine
hypotheses. 

In [9]:
pottery.drop('No', axis=1).groupby('Kiln').aggregate([np.mean, np.std])

Al2O3               Fe2O3                 MgO                 CaO  \
           mean       std      mean       std      mean       std      mean   
Kiln                                                                          
1     16.940909  1.510490  7.430909  0.652416  1.836364  0.203997  0.942273   
2     12.564286  1.377069  6.372143  0.785564  4.826429  1.088221  0.202143   
3     11.700000  0.141421  5.415000  0.035355  3.855000  0.120208  0.295000   
4     18.180000  1.775387  1.712000  0.435970  0.674000  0.032094  0.026000   
5     17.320000  1.658915  1.512000  0.736016  0.606000  0.063482  0.052000   

                    Na2O                 K2O                TiO2            \
           std      mean       std      mean       std      mean       std   
Kiln                                                                         
1     0.285289  0.348182  0.159840  3.105455  0.219691  0.896364  0.201767   
2     0.058201  0.250714  0.122629  3.927857  1.012713  0.706429  0.061844   
3     0.007071  0.050000  0.014142  4.575000  0.091924  0.575000  0.021213   
4     0.026077  0.054000  0.027928  2.076000  0.177003  1.046000  0.265575   
5     0.034205  0.048000  0.010954  1.966000  0.195269  0.994000  0.212908   

           MnO                 BaO            
          mean       std      mean       std  
Kiln                                          
1     0.071727  0.018419  0.017136  0.002587  
2     0.144500  0.079235  0.017000  0.004000  
3     0.097500  0.017678  0.014000  0.001414  
4     0.002200  0.002168  0.016400  0.003286  
5     0.004200  0.002387  0.015600  0.002793

In [10]:
pottery = pottery[~(pottery.Kiln == 3)]

f_stats = []
p_vals = []

kiln1 = pottery[pottery.Kiln == 1]
kiln2 = pottery[pottery.Kiln == 2]
kiln4 = pottery[pottery.Kiln == 4]
kiln5 = pottery[pottery.Kiln == 5]

minerals = ['Al2O3', 'Fe2O3', 'MgO', 'CaO', 'Na2O', 'K2O','TiO2', 'MnO', 'BaO']

for m in minerals:
    f, p = stats.f_oneway(kiln1[m], kiln2[m], kiln4[m], kiln5[m])
    f_stats.append(f)
    p_vals.append(p)
    
results = pd.DataFrame({'F': f_stats, 'p': p_vals}, index=minerals)

In [11]:
results

,F,p
Al2O3,31.306414,8.602438e-11
Fe2O3,167.892554,2.075886e-23
MgO,103.188037,2.075129e-19
CaO,60.851910,2.448593e-15
Na2O,11.561208,1.174614e-05
K2O,20.343030,2.692834e-08
TiO2,6.351567,1.193501e-03
MnO,18.716849,7.338810e-08
BaO,0.362216,7.805878e-01


2 . At significance level $\alpha=0.05$, apply the Bonferroni
correction method to test $H_{0}: \mu_1=\mu_2=\mu_3=\mu_4=\mu_5$.
Summarize your conclusions.  

In [12]:
def bonferroni(alpha, m, results):
    adj_alpha = alpha / m
    return results.assign(reject=results.p <= adj_alpha)

bonferroni(0.05, 9, results)

,F,p,reject
Al2O3,31.306414,8.602438e-11,True
Fe2O3,167.892554,2.075886e-23,True
MgO,103.188037,2.075129e-19,True
CaO,60.851910,2.448593e-15,True
Na2O,11.561208,1.174614e-05,True
K2O,20.343030,2.692834e-08,True
TiO2,6.351567,1.193501e-03,True
MnO,18.716849,7.338810e-08,True
BaO,0.362216,7.805878e-01,False


3 . Apply the Benjamini-Hochberg method to test the nine
hypotheses $H_{0}: \mu_{1k}=\mu_{2k}=\mu_{3k}=\mu_{4k}=\mu_{5k},k=1,...,9$,
simultaneously with FDR controlled at $\alpha=0.05$. Summarize
your conclusions.

In [42]:
def bh_correct(alpha, m, results):
    sorted_p = sorted(results.p)
    ranks = np.arange(1, m+1)
    
    adj_alphas = (ranks * alpha) / m
    rejected = sorted_p <= adj_alphas
    BH_thresh = max(adj_alphas[rejected == True])
    return BH_thresh, results.assign(reject=results.p <= BH_thresh)

BH_thresh, out = bh_correct(0.05, 9, results)

In [43]:
BH_thresh

0.044444444444444446

In [44]:
out

,F,p,reject
Al2O3,31.306414,8.602438e-11,True
Fe2O3,167.892554,2.075886e-23,True
MgO,103.188037,2.075129e-19,True
CaO,60.851910,2.448593e-15,True
Na2O,11.561208,1.174614e-05,True
K2O,20.343030,2.692834e-08,True
TiO2,6.351567,1.193501e-03,True
MnO,18.716849,7.338810e-08,True
BaO,0.362216,7.805878e-01,False
